In [1]:
from openai import OpenAI
import getpass
from elasticsearch import Elasticsearch


## Q1. Running Elastic

In [71]:
!curl localhost:9200

{
  "name" : "7890b070eaaf",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "C0_-ymNMSMC-FNPstdnqQQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


python(22525) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


### Getting the Data 

In [3]:
import requests

In [4]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [67]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Q2. Indexing the Data 

In [65]:
es_client = Elasticsearch('http://localhost:9200') 

In [66]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [8]:
from tqdm.auto import tqdm

In [68]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

In [10]:
query = "How do I execute a command in a running docker container?"

In [69]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        print(hit, hit['_score'])
        result_docs.append(hit['_source'])
    
    return result_docs

In [70]:
elastic_search(query)

{'_index': 'course-questions', '_id': 'Dv6gZZABBBp9DYP5tWP-', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}} 84.050095
{'_index': 'course-questions', '_id': 'HP6gZZABBBp9DYP5q2Hd', '_score': 75.54128, '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debia

## Q4. Filtering

In [72]:
def elastic_search_with_filtering(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [73]:
search_results = elastic_search_with_filtering(query)

In [20]:
search_results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

## Q5. Building a prompt 

In [57]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """
    context = ""
    
    for doc in search_results:
        context = context + f"Q:{doc['question']}\nA:{doc['text']}\n\n"

    context = context.strip()
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt
    

In [58]:
prompt = build_prompt(query, search_results)
len(prompt)

1472

In [61]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: How do I execute a command in a running docker container?

    CONTEXT:
    Q:How do I debug a docker container?
A:Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q:How do I copy files from my local machine to docker container?
A:You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory c

## Q6. Tokens

In [29]:
import tiktoken

In [30]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [33]:
len(encoding.encode(prompt))

326

In [62]:
tokens = encoding.encode("I am building a RAG")
print(f"tokens={tokens}, token count = {len(tokens)}")

decoded = encoding.decode(tokens)
print(f"decoded={decoded}")


tokens=[40, 939, 6282, 261, 460, 2971], token count = 6
decoded=I am building a RAG


## Bonus: generating the answer

In [34]:
from openai import OpenAI

In [37]:
api_key = getpass.getpass('Enter your OpenAI API key:')

In [40]:
client = OpenAI(api_key=api_key)

In [41]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [42]:
def rag(query): 
    search_results = elastic_search_with_filtering(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [75]:
query

'How do I execute a command in a running docker container?'

In [81]:
response = rag(query)
print(response)

To execute a command in a running Docker container, you can use the `docker exec` command. First, you need to find the container ID using `docker ps`. Then, use the following command to execute a command in the specific container:

```bash
docker exec -it <container-id> <command>
```

For example, if you want to start a bash session inside the container, you would use:

```bash
docker exec -it <container-id> bash
```


## Bonus: calculating the costs 

In [78]:
cost_per_input_token = 0.005/1000
cost_per_output_token = 0.015/1000

In [79]:
input_token_count = 150
output_token_count = 250

In [80]:
cost_per_1000_requests = input_token_count * cost_per_input_token * 1000 + output_token_count * cost_per_output_token * 1000
print(f"cost for 1000 requests with  {input_token_count}  tokens in the prompt and {output_token_count} in response is ${cost_per_1000_requests}") 

cost for 1000 requests with  150  tokens in the prompt and 250 in response is $4.5


In [83]:
input_token_count = len(encoding.encode(prompt))
output_token_count = len(encoding.encode(response))
print(f"Token Counts for query -> \"{query}\" input = {input_token_count}, output = {output_token_count}")

Token Counts for query -> "How do I execute a command in a running docker container?" input = 326, output = 98


In [84]:
cost_per_1000_requests = input_token_count * cost_per_input_token * 1000 + output_token_count * cost_per_output_token * 1000
print(f"cost for 1000 requests with  {input_token_count}  tokens in the prompt and {output_token_count} in response is ${cost_per_1000_requests}") 

cost for 1000 requests with  326  tokens in the prompt and 98 in response is $3.1
